In [33]:
import pandas as pd
from typing import List
import geopy.distance

# Arrival

In [61]:
arrival_df = pd.read_excel('Data/arrivals.xlsx', sheet_name=1)
arrival_df.head(1)

,Vessel Name,Port Call Type,Port Type,Port At Call,Port At Call Country,Latitude,Longitude,Ata/atd,Time At Port,Port Country,Port Location,Type,Remarks,Latitude (generated),Longitude (generated),Total Capcity of Methanol Storage ( In Metric Tonnes)
0,SEABOURN VENTURE,DEPARTURE,Port,REYKJAVIK,IS,64.1265,-21.8174,2023-06-04 20:07:00,14h 19m,China,Zhuhai,Confirmed Methanol Supply / Storage,NaN,22.2667,113.5667,3957.0


In [66]:
# Preprocess Arrivals df
arrival_df.rename(columns={
    'Port At Call': 'location', 
    'Port At Call Country': 'country',
    'Latitude': 'lat',
    'Longitude': 'long',
    'Ata/atd': 'time'
    }, inplace=True)
arrival_df['location']=arrival_df.location.map(lambda y: y.title())
arrival_df = arrival_df [['location', 'country', 'lat', 'long', 'time']]
arrival_df.head(1)

,location,country,lat,long,time
0,Reykjavik,IS,64.1265,-21.8174,2023-06-04 20:07:00


# Methanol

In [19]:
methanol_df = pd.read_excel('Data/emethanol.xlsx')
methanol_df.head(1)

,Latitude,Longtitude,Capacity (t/y),City,Company,Country,Feedstock
0,63.868877,20.411105,100000.0,Umeå,Liquid Wind and Umeå Energi,Sweden,CO2 and H2 from water electrolysis


In [20]:
# Preprocess methanol df
methanol_df.rename(columns={
    'City': 'location', 
    'Country': 'country',
    'Latitude': 'lat',
    'Longtitude': 'long',
    }, inplace=True)
methanol_df = methanol_df [['location', 'country', 'lat', 'long']]
methanol_df.head(1)

,location,country,lat,long
0,Umeå,Sweden,63.868877,20.411105


# Compute

In [58]:
def compute_distance(A: pd.DataFrame, B: pd.DataFrame, max_distance: int) -> pd.DataFrame:
    result = pd.DataFrame()
    for index, Arow in A.iterrows():
        for index, Brow in B.iterrows():
            coords_1 = (Arow.lat, Arow.long)
            coords_2 = (Brow.lat, Brow.long)
            dist=(geopy.distance.geodesic(coords_1, coords_2))
            if dist.km<= max_distance:
                new_row = {}
                new_row['arrival_location'] = Arow.location
                new_row['arrival_country'] = Arow.country
                new_row['methanol_location'] = Brow.location
                new_row['methanol_country'] = Brow.country
                new_row['km_distance'] = round(dist.km, 1)
                new_row['nautical_distance'] = round(dist.nautical, 1)
                row_df = pd.DataFrame(new_row, index=[0])
                result = pd.concat([result, row_df])
    return result.reset_index(drop=True)

In [80]:
def remove_duplicates(df: pd.DataFrame, column: str = 'km_distance') -> pd.DataFrame:
    return df.sort_values(by=[column], ascending=True).drop_duplicates(subset='arrival_location').sort_index().reset_index(drop=True)

In [78]:
res=compute_distance(arrival_df, methanol_df, 150)
res

,arrival_location,arrival_country,methanol_location,methanol_country,km_distance,nautical_distance
0,Reykjavik,IS,Grindavík,Iceland,41.1,22.2
1,Grundarfjordur,IS,Grindavík,Iceland,124.2,67.1
2,Siglufjordur,IS,Húsavík,Iceland,72.0,38.9
3,Invergordon,GB,Nigg,Scotland,141.9,76.6
4,Leith,GB,Nigg,Scotland,146.0,78.8
5,Leith,GB,Grangemouth,UK,35.0,18.9
6,Clydeportgreenock,GB,Grangemouth,UK,65.8,35.5
7,Douglasanch,IM,Eastham,UK,136.5,73.7
8,Vestmannaeyjar,IS,Grindavík,Iceland,118.1,63.8
9,Reykjavik,IS,Grindavík,Iceland,41.1,22.2


In [81]:
res1=remove_duplicates(res)
res1

,arrival_location,arrival_country,methanol_location,methanol_country,km_distance,nautical_distance
0,Grundarfjordur,IS,Grindavík,Iceland,124.2,67.1
1,Siglufjordur,IS,Húsavík,Iceland,72.0,38.9
2,Invergordon,GB,Nigg,Scotland,141.9,76.6
3,Leith,GB,Grangemouth,UK,35.0,18.9
4,Clydeportgreenock,GB,Grangemouth,UK,65.8,35.5
5,Douglasanch,IM,Eastham,UK,136.5,73.7
6,Vestmannaeyjar,IS,Grindavík,Iceland,118.1,63.8
7,Akureyri,IS,Húsavík,Iceland,53.1,28.7
8,Grundarfjorduranch,IS,Grindavík,Iceland,124.2,67.1
9,Reykjavik,IS,Grindavík,Iceland,41.1,22.2
